In [75]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import tree
from sklearn.metrics import *
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor

In [2]:
train=pd.read_csv("train.csv")
test=pd.read_csv("test.csv")
submission=pd.read_csv("sample_submission.csv")

In [3]:
train.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


In [4]:
train.isna().sum()

Item_Identifier                 0
Item_Weight                  1463
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  2410
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
dtype: int64

In [5]:
print(train.shape)
print(test.shape)

(8523, 12)
(5681, 11)


In [6]:
test.isna().sum()

Item_Identifier                 0
Item_Weight                   976
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  1606
Outlet_Location_Type            0
Outlet_Type                     0
dtype: int64

In [7]:
test.isna().sum()

Item_Identifier                 0
Item_Weight                   976
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  1606
Outlet_Location_Type            0
Outlet_Type                     0
dtype: int64

In [8]:
test.Item_Weight=test.Item_Weight.fillna(test.Item_Weight.mean())
test.Outlet_Size=test.Outlet_Size.fillna(test.Outlet_Size.mode()[0])

In [9]:
test.isna().sum()

Item_Identifier              0
Item_Weight                  0
Item_Fat_Content             0
Item_Visibility              0
Item_Type                    0
Item_MRP                     0
Outlet_Identifier            0
Outlet_Establishment_Year    0
Outlet_Size                  0
Outlet_Location_Type         0
Outlet_Type                  0
dtype: int64

In [10]:
Q1 = train.quantile(0.25)
Q3 = train.quantile(0.75)
IQR = Q3 - Q1
print(IQR)

Item_Weight                     8.076250
Item_Visibility                 0.067596
Item_MRP                       91.817200
Outlet_Establishment_Year      17.000000
Item_Outlet_Sales            2267.049000
dtype: float64


In [11]:
train = train[~((train < (Q1 - 1.5 * IQR)) |(train > (Q3 + 1.5 * IQR))).any(axis=1)]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Automatic reindexing on DataFrame vs Series comparisons is deprecated and will raise ValueError in a future version.  Do `left, right = left.align(right, axis=1, copy=False)` before e.g. `left == right`
  """Entry point for launching an IPython kernel.


In [12]:
train.shape

(8193, 12)

In [13]:
train.Item_Weight=train.Item_Weight.fillna(train.Item_Weight.mean())
train.Outlet_Size=train.Outlet_Size.fillna(train.Outlet_Size.mode()[0])

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [14]:
train.isna().sum()

Item_Identifier              0
Item_Weight                  0
Item_Fat_Content             0
Item_Visibility              0
Item_Type                    0
Item_MRP                     0
Outlet_Identifier            0
Outlet_Establishment_Year    0
Outlet_Size                  0
Outlet_Location_Type         0
Outlet_Type                  0
Item_Outlet_Sales            0
dtype: int64

In [15]:
train.shape

(8193, 12)

In [16]:
train.describe()

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Item_Outlet_Sales
count,8193.000000,8193.000000,8193.000000,8193.000000,8193.000000
mean,12.865099,0.063221,139.104030,1998.089711,2089.540121
std,4.266982,0.046285,61.536438,8.286946,1501.498356
min,4.555000,0.000000,31.290000,1985.000000,33.290000
25%,9.300000,0.026818,92.980400,1987.000000,857.550400
50%,12.865099,0.052729,141.147000,1999.000000,1783.678200
75%,16.100000,0.092783,183.663400,2004.000000,3020.068800
max,21.350000,0.195721,266.888400,2009.000000,6478.234000


In [17]:
train.dtypes

Item_Identifier               object
Item_Weight                  float64
Item_Fat_Content              object
Item_Visibility              float64
Item_Type                     object
Item_MRP                     float64
Outlet_Identifier             object
Outlet_Establishment_Year      int64
Outlet_Size                   object
Outlet_Location_Type          object
Outlet_Type                   object
Item_Outlet_Sales            float64
dtype: object

In [18]:
train.Item_Outlet_Sales=train.Item_Outlet_Sales.astype(str)

In [19]:
col_list = []
for col in train.columns:
    if ((train[col].dtype == 'object') & (col != 'Item_Outlet_Sales')):
        col_list.append(col)

In [20]:
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
for i in col_list:
    train[i]=labelencoder.fit_transform(train[i])

In [21]:
train.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,156,9.30,1,0.016047,4,249.8092,9,1999,1,0,1,3735.138
1,8,5.92,2,0.019278,14,48.2692,3,2009,1,2,2,443.4228
2,662,17.50,1,0.016760,10,141.6180,9,1999,1,0,1,2097.27
3,1121,19.20,2,0.000000,6,182.0950,0,1998,1,2,0,732.38
4,1297,8.93,1,0.000000,9,53.8614,1,1987,0,2,1,994.7052


In [48]:
list_col = []
for col in test.columns:
    if (test[col].dtype == 'object') :
        list_col.append(col)

In [49]:
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
for i in list_col:
    test[i]=labelencoder.fit_transform(test[i])

In [50]:
test.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type
0,1103,20.750000,1,0.007565,13,107.8622,9,1999,1,0,1
1,1067,8.300000,4,0.038428,4,87.3198,2,2007,1,1,1
2,1406,14.600000,1,0.099575,11,241.7538,0,1998,1,2,0
3,809,7.315000,1,0.015388,13,155.0340,2,2007,1,1,1
4,1184,12.695633,2,0.118599,4,234.2300,5,1985,1,2,3


In [23]:
x=train.drop(['Item_Outlet_Sales'], axis=1)
y=train.Item_Outlet_Sales

In [24]:
x_train,x_test, y_train, y_test=train_test_split(x,y,random_state=0)

In [25]:
x_test

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type
3925,1054,18.200000,1,0.012597,13,141.2180,1,1987,0,2,1
4905,1363,20.100000,1,0.077454,8,59.2536,8,1997,2,0,1
2082,231,8.750000,2,0.074743,6,187.6556,9,1999,1,0,1
6527,66,5.980000,0,0.075711,7,55.6614,7,2002,1,1,1
6747,966,12.865099,1,0.132275,0,123.8388,4,1985,2,0,0
...,...,...,...,...,...,...,...,...,...,...,...
2847,176,16.000000,2,0.099189,5,88.9856,1,1987,0,2,1
4497,1376,14.650000,1,0.072179,9,262.9594,9,1999,1,0,1
3962,2,8.270000,2,0.127821,14,185.9924,1,1987,0,2,1
4400,762,4.555000,2,0.034411,5,111.3544,9,1999,1,0,1


In [26]:
y_test

3925     2936.178
4905     735.0432
2082    5257.1568
6527     331.5684
6747     123.8388
          ...    
2847    1669.8264
4497     5233.188
3962     555.2772
4400    1342.2528
3910    1621.8888
Name: Item_Outlet_Sales, Length: 2049, dtype: object

In [52]:
submission.head()

,Item_Identifier,Outlet_Identifier,Item_Outlet_Sales
0,FDW58,OUT049,1000
1,FDW14,OUT017,1000
2,NCN55,OUT010,1000
3,FDQ58,OUT017,1000
4,FDY38,OUT027,1000


In [61]:
clf=tree.DecisionTreeClassifier()
clf.fit(x_train, y_train)

DecisionTreeClassifier()

In [63]:
y_test_predict=clf.predict(x_test)
y_test_predict

array(['3387.5904', '858.882', '1491.392', ..., '2406.2012', '663.1368',
       '1621.8888'], dtype=object)

In [71]:
r2_on_test=r2_score(y_test, y_test_predict)
r2_on_test

0.09751818356127173

In [70]:
import numpy as np
print("RMSE : %.4g" % np.sqrt(mean_squared_error(y_test, y_test_predict)))

RMSE : 1420


In [64]:
y_predict=clf.predict(test)

In [72]:
submission['Item_Outlet_Sales']=y_predict
submission.head()

,Item_Identifier,Outlet_Identifier,Item_Outlet_Sales
0,FDW58,OUT049,982.7208
1,FDW14,OUT017,1062.6168
2,NCN55,OUT010,484.7024
3,FDQ58,OUT017,3755.112
4,FDY38,OUT027,5672.616


In [74]:
submission.to_csv("submission_test.csv",index=False)

In [78]:
num_folds = 10
param_grid = dict(n_estimators=np.array([80,82,84,85,90,91,92,94,95,150,200]))
RMS = 'neg_mean_squared_error'
model = GradientBoostingRegressor()
kfold = KFold(n_splits=num_folds)
grid = GridSearchCV(estimator=model, param_grid=param_grid, scoring=RMS, cv=kfold)
grid_result = grid.fit(x_train, y_train)
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: -952446.752795 using {'n_estimators': 80}
-952446.752795 (65384.616153) with: {'n_estimators': 80}
-952636.733380 (65526.944897) with: {'n_estimators': 82}
-952645.724674 (64638.812566) with: {'n_estimators': 84}
-952868.369650 (65127.512090) with: {'n_estimators': 85}
-952616.382398 (65083.894787) with: {'n_estimators': 90}
-952560.012127 (64486.337064) with: {'n_estimators': 91}
-952836.470090 (64614.754145) with: {'n_estimators': 92}
-953400.503370 (64581.276435) with: {'n_estimators': 94}
-953479.257866 (64286.334051) with: {'n_estimators': 95}
-963230.897800 (66717.252278) with: {'n_estimators': 150}
-973627.668653 (70082.289146) with: {'n_estimators': 200}


In [81]:
model = GradientBoostingRegressor(alpha=0.9,learning_rate=0.05, max_depth=5, min_samples_leaf=3, min_samples_split=2, n_estimators=80, random_state=30)
model.fit(x_train, y_train)
# transform the validation dataset
predictions = model.predict(x_train)
print("RMSE = ", mean_squared_error(y_train, predictions))

RMSE =  844206.0661615897


In [82]:
y_pred=model.predict(test)

In [83]:
submission['Item_Outlet_Sales']=y_pred
submission.head()

,Item_Identifier,Outlet_Identifier,Item_Outlet_Sales
0,FDW58,OUT049,1701.741246
1,FDW14,OUT017,1399.579552
2,NCN55,OUT010,630.987214
3,FDQ58,OUT017,2497.297752
4,FDY38,OUT027,4560.381580


In [86]:
submission.to_csv("submission_test.csv",index=False)